# pyiron workflow

This code is in development stage, and will change considerably..

In [ ]:
from pyiron_contrib.workflow import Workflow
from pyiron import Project

In [ ]:
@Workflow.wrap_as.single_value_node(update_on_instantiation=False)
def calculate_elastic_matrix(
    structure,
):
    pr = Project("elastic_constant")
    
    job = pr.create.job.Lammps(job_name="lammps_job")
    job.structure = structure
    job.potential = '2009--Mendelev-M-I--Al-Mg--LAMMPS--ipr1'
    job.calc_minimize(pressure=0)
    
    elastic_job = job.create_job(pr.job_type.ElasticTensor, "elastic_job")
    elastic_job.input["eps_range"] = 0.001
    elastic_job.run()
    
    elastic_matrix = elastic_job["output/elastic_tensor"]
    
    pr.remove_jobs(silently=True, recursive=True)
    pr.remove(enable=True)
    
    return elastic_matrix

In [ ]:
wf = Workflow("youngs_modulus")
wf.structure = wf.add.atomistics.BulkStructure(cubic=True)
wf.calc = calculate_elastic_matrix(wf.structure, project_name=wf.label)

In [ ]:
wf(structure_element="Al")

In [ ]:
wf.outputs.calc_elastic_matrix